In [30]:
#Indicative snapsot of how the Optuna Framework was used in this project. 
#Several experiments took place with different combination of layers (mainly
#Convolutional and Dense) and different hyperparameter values. 

In [31]:
def fit_evaluate_model(params: dict) -> float:
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)


    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2, decay_steps=80000, decay_rate=0.9)
    
    X_train, y_train, X_test, y_test, X_val, y_val = get_data()
    inputShape = np.array(X_train).shape[1], np.array(X_train).shape[2]
    
    ## architecture
    model = Sequential()
    model.add(Conv1D(filters=104, kernel_size=3, activation='relu', input_shape=inputShape, padding='same'))
    for i in range(params['layers']):
        model.add(Conv1D(filters=params['filters'], kernel_size=params['kernel'], activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(150))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(150))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(150))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(24, activation='relu'))
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mse')

    # fit & evaluation
    model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val), np.array(y_val)), epochs=200, callbacks=[callback], batch_size=1024)

    predictions_per_country = []
    inversed_y_per_country = []
    results = []
    for i in range(len(scalers)):
        predict = model.predict(np.array(X_test[i]))
        if np.isnan(predict).any():
            print(i)
            continue;
        predictions_per_country.append(scalers[i].inverse_transform(predict))
        inversed_y_per_country.append(scalers[i].inverse_transform(y_test[i]))
        results.append(mean_squared_error(np.array(inversed_y_per_country[i]), predictions_per_country[i], squared=False))
    if(results==0):
        return 1000000
    else:
        return (sum(results)/len(results)) #returns the average MSE of all countries

In [ ]:
def objective(trial):
    
    layers = trial.suggest_int('layers', 3, 10, 3)
    kernel = trial.suggest_int('kernel', 2, 6, 2)
    filters = trial.suggest_int('filters', 8, 128, 32)

    params = {
        'layers': layers,
        'kernel': kernel,
        'filters': filters,
    }
    return fit_evaluate_model(params)


class SaveStudyCallback:
    def __init__(self):
        pass

    def __call__(self, study: optuna.study.Study, trial:     optuna.trial.FrozenTrial) -> None:
        joblib.dump(study, 'study2')
        

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[SaveStudyCallback()])

In [46]:
study = joblib.load('study2')
study.trials 
df = study.trials_dataframe()
df.sort_values(by=['value'])

,number,value,datetime_start,datetime_complete,duration,params_decay_steps,state
4,4,198.670727,2023-01-17 18:26:04.926918,2023-01-17 18:36:19.081038,0 days 00:10:14.154120,90000,COMPLETE
29,29,222.721799,2023-01-17 20:15:46.748896,2023-01-17 20:30:46.774394,0 days 00:15:00.025498,60000,COMPLETE
9,9,230.523894,2023-01-17 18:52:53.689215,2023-01-17 18:57:48.185461,0 days 00:04:54.496246,30000,COMPLETE
19,19,254.488613,2023-01-17 19:26:12.514817,2023-01-17 19:35:55.073424,0 days 00:09:42.558607,40000,COMPLETE
10,10,261.496417,2023-01-17 18:57:48.190120,2023-01-17 19:08:01.505387,0 days 00:10:13.315267,60000,COMPLETE
11,11,283.971979,2023-01-17 19:08:01.509966,2023-01-17 19:14:22.043560,0 days 00:06:20.533594,70000,COMPLETE
26,26,303.549475,2023-01-17 19:53:52.597585,2023-01-17 20:02:40.949204,0 days 00:08:48.351619,80000,COMPLETE
22,22,304.137775,2023-01-17 19:39:23.935676,2023-01-17 19:46:00.078304,0 days 00:06:36.142628,30000,COMPLETE
6,6,305.345648,2023-01-17 18:37:51.753120,2023-01-17 18:49:32.943837,0 days 00:11:41.190717,110000,COMPLETE
12,12,402.828621,2023-01-17 19:14:22.049284,2023-01-17 19:16:10.076621,0 days 00:01:48.027337,10000,COMPLETE


In [ ]:
#The final three architectures arose after taking into consideration the
#best trials' results from dozens of Optuna tests. 